# Hometask 3

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
X

array([[1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00,
        1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00],
       [1.00e+00, 1.00e+00, 2.00e+00, 5.00e+00, 3.00e+00, 0.00e+00,
        5.00e+00, 1.00e+01, 1.00e+00, 2.00e+00],
       [5.00e+02, 7.00e+02, 7.50e+02, 6.00e+02, 1.45e+03, 8.00e+02,
        1.50e+03, 2.00e+03, 4.50e+02, 1.00e+03],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 2.00e+00, 1.00e+00,
        3.00e+00, 3.00e+00, 1.00e+00, 2.00e+00]])

In [4]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [5]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [6]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  5.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [7]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    err = np.sum(err)
    return err

In [8]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss(y1, y_pred1)

0.10536051565782628

In [9]:
# Плохой пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0, 0.1])
calc_logloss(y1, y_pred1)

/tmp/ipykernel_153734/2163466517.py:2: RuntimeWarning: divide by zero encountered in log
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))


inf

In [10]:
# Плохой пример применения 2
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

/tmp/ipykernel_153734/2163466517.py:2: RuntimeWarning: divide by zero encountered in log
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
/tmp/ipykernel_153734/2163466517.py:2: RuntimeWarning: invalid value encountered in multiply
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))


nan

In [11]:
def calc_logloss(y, y_pred):
    y_pred = np.clip(y_pred, 1e-8, 1 - 1e-8)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    err = np.sum(err)
    return err

In [12]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss(y1, y_pred1)

0.10536051565782628

Сходятся все знаки

In [13]:
# Плохой пример применения -> норма
y1 = np.array([1, 0])
y_pred1 = np.array([0, 0.1])
calc_logloss(y1, y_pred1)

9.263020629805096

In [14]:
# Плохой пример применения 2 -> норма
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.052680262828913194

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [15]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [16]:
def eval_model(X, y, iterations, alpha=1e-4, debug=False):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    prev_err = float('inf')
    err = float('inf')
    i = 0
    
    for i in range(0, iterations):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        # debug
        if debug and (i + 1) % (iterations / 10) == 0:
            print(i, W, err)
        
        if abs(prev_err - err) < 1e-8:
            break
        prev_err = err
        
    return W, err, i + 1

In [17]:
W, logloss_err, iters = eval_model(X_st, y, iterations=5000, alpha=1e-4, debug=True)

499 [ 0.47822106 -0.20918419  0.64605995  1.49531432] 1.0559639161585663
999 [ 0.46074241 -0.27647603  0.64395686  1.46891696] 0.945214871179169
1499 [ 0.44435675 -0.33951515  0.64152026  1.44406912] 0.8478360281905125
1999 [ 0.42913461 -0.39751872  0.63899625  1.42106251] 0.7651177745022372
2499 [ 0.41513888 -0.44956784  0.63671032  1.40023024] 0.6981293270105478
2999 [ 0.40240136 -0.49488241  0.63498247  1.38184975] 0.6468813805442176
3499 [ 0.39088925 -0.53321212  0.63401986  1.36601535] 0.6097021563515638
3999 [ 0.38049869 -0.56497213  0.63387116  1.35259671] 0.5836774145473647
4499 [ 0.37108342 -0.59102182  0.63446603  1.3413119 ] 0.5657147376221167
4999 [ 0.36249064 -0.61234585  0.63568321  1.33183158] 0.5532737788236778


In [18]:
%%time
logloss_error_min = float('inf')

for i, alpha in enumerate(np.linspace(1e-10, 40, 100)):
    W, logloss_err, iters = eval_model(X_st, y, iterations=10000, alpha=alpha)
    if logloss_err < logloss_error_min:
        logloss_error_min = logloss_err
        params = (iters, alpha)
    print('\rIter = {}'.format(i), end="")
    
print('\nLogloss error = {}, iterations = {}, alpha = {}'.format(logloss_error_min, params[0], params[1]))

Iter = 99
Logloss error = 0.0030241982880477533, iterations = 10000, alpha = 40.0
CPU times: user 43.9 s, sys: 589 ms, total: 44.5 s
Wall time: 43.6 s


In [19]:
W, logloss_err, iters = eval_model(X_st, y, iterations=10000, alpha=40, debug=True)

999 [-252.70534998  -34.84963519  -69.25975799  216.16794982] 0.005624636115151258
1999 [-256.07679479  -32.70169793  -71.04494311  215.55070883] 0.005135494617619681
2999 [-259.16691099  -30.73551928  -72.68007348  214.98853016] 0.004725101861037168
3999 [-262.02164852  -28.91917915  -74.19058086  214.46932094] 0.004374897786488858
4999 [-264.67413501  -27.23150616  -75.59404361  213.98691353] 0.004072580490102784
5999 [-267.15090509  -25.655618    -76.90451063  213.53647252] 0.0038090069998965
6999 [-269.47361436  -24.1777398   -78.13344556  213.11405698] 0.003577215144847818
7999 [-271.66016625  -22.78648335  -79.29032524  212.71640926] 0.0033718107888596255
8999 [-273.72552844  -21.47232843  -80.38307272  212.34080634] 0.003188551096868604
9999 [-275.68171953  -20.22883246  -81.41877552  211.98549957] 0.0030241982880477533


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred_proba.

In [20]:
def calc_pred_proba(W, X):
    z = np.dot(W, X)
    return 1/(1 + np.exp(-z))

In [21]:
W

array([-275.68171953,  -20.22883246,  -81.41877552,  211.98549957])

In [22]:
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  5.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [23]:
calc_pred_proba(W, X_st)

array([1.52462297e-02, 4.03092303e-17, 1.00000000e+00, 5.70871559e-45,
       9.99608757e-01, 1.25446823e-15, 1.00000000e+00, 7.36953079e-07,
       9.85619731e-01, 1.00000000e+00])

4. Создайте функцию calc_pred, возвращающую предсказанный класс. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred.

In [24]:
def calc_pred(W, X, treshold = 0.5):
    y_pred = calc_pred_proba(W, X)
    y_pred = np.where(y_pred > treshold, 1, 0 )
    return y_pred

In [25]:
y_pred = calc_pred(W, X_st)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [26]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [27]:
accuracy = np.sum(y_pred)/y_pred.shape[0] # accuracy
accuracy

0.5

In [28]:
err_matrix = np.array(pd.crosstab(y, y_pred)) # матрица ошибок
err_matrix

array([[5, 0],
       [0, 5]])

In [29]:
precition = err_matrix[1,1]/(err_matrix[1,1] + err_matrix[0,1]) # точность 
precition

1.0

In [30]:
recall = err_matrix[1,1]/(err_matrix[1,1] + err_matrix[1,0]) # полнота
recall

1.0

In [31]:
F = 2 * precition * recall / (precition + recall)
F

1.0

6. Могла ли модель переобучиться? Почему?

Могла (мало данных, сильная корреляция, очень большой коэффициент alpha).